In [1]:
%run StdPackages.ipynb
os.chdir(d['py'])
from IOfunctions import *
os.chdir(d['curr'])

No clean-up of work-folder


*Load IO:*

In [2]:
name = 'GR18'
IO_file = os.path.join(d['data'], 'IO2018_v.xlsx')
IO_file_inv = os.path.join(d['data'],'IO2018_I.xlsx')
wb = read.simpleLoad(IO_file)
wb_inv = read.simpleLoad(IO_file_inv)

Database:

In [3]:
db = GpyDB(**{'name': f"IO_{name}"})

## 1. Extract IO value blocks:

In [4]:
df = pd.DataFrame(wb['IO'].values)
rowCategories, colCategories = df.iloc[:,0], df.iloc[0,:]
rowIndex, colIndex = df.iloc[:,0], df.iloc[2,:]

In [5]:
_stdOffset = {'row0': 1, 'rowE': -1, 'col0': 0, 'colE': -1}
rowMarkers = {'P': {'ref': 'Dansk produktion','offset': {}},
              'M': {'ref': 'Import', 'offset': {}},
              'OT': {'ref': 'Andre udenlandske transaktioner', 'offset': {}},
              'PI': {'ref': 'Primære inputs', 'offset': {}},
              'TI': {'ref': 'Input/ endelig anvendelse i køberpriser', 'offset': {}},
              'PV': {'ref': 'Produktionsværdi', 'offset': {}}
             }
colMarkers = {'In': {'ref': 'Input i produktionen (Transaktionskode 2000)', 'offset': {'colE': -2}},
              'C' : {'ref': 'Privat forbrug (Transaktions-kode 3110)', 'offset': {'colE': -1}},
              'G_NPISH': {'ref': 'NPISH (Transaktionskode 3130)', 'offset': {}},
              'G_MVPC' : {'ref': 'Markedsmæssigt individuelt offentligt forbrug (Transaktionskode 3141)', 'offset': {}},
              'G_NMVPC': {'ref': 'Ikke markedsmæssigt individuelt offentligt forbrug (Transaktionskode 3142)', 'offset': {}},
              'G_CPC':   {'ref': 'Kollektivt offentligt forbrug (Transaktionskode 3200)', 'offset': {}},
              'I': {'ref': 'Faste bruttoinvesteringer', 'offset': {}},
              'Other': {'ref': 'Andre Anvendelser', 'offset':{}},
              'T': {'ref': 'Total'}
             }
rKeys,rVals = list(rowMarkers),list(rowMarkers.values())
cKeys,cVals = list(colMarkers),list(colMarkers.values())
taxCategories = ['Produktskatter og subsidier, netto', 'Moms', 'Andre produktionsskatter', 'Andre produktionssubsidier']
wageCategory = 'Aflønning af ansatte'
residualIncomeCategory = 'Overskud af produktionen og blandet indkomst'
itoryCategories = ['5300','5200']
exportCategory = '6000'

In [6]:
def getLoc(x, loc):
    return x[x==loc].index[0]
def getOffset(r, c):
    return _stdOffset | r['offset'] | c['offset']
def getLocs(r = None, rr = None, c = None, cc = None, rowCategories = None, colCategories = None):
    o = getOffset(r, c)
    return [(getLoc(rowCategories, r['ref'])+o['row0'], getLoc(rowCategories, rr['ref'])+o['rowE']+1), 
            (getLoc(colCategories, c['ref'])+o['col0'], getLoc(colCategories, cc['ref'])+o['colE']+1)]
def extractBlock(df, locs):
    return df.iloc[locs[0][0]:locs[0][1], locs[1][0]:locs[1][1]]

*Extract location of block splits and extract blocks*

In [7]:
locs = {f"{rKeys[i]}/{cKeys[j]}": getLocs(r=rVals[i], rr = rVals[i+1], c = cVals[j], cc = cVals[j+1], rowCategories = rowCategories, colCategories = colCategories)
        for j in range(len(cKeys)-1) for i in range(len(rKeys)-1)}
blocks = {k: extractBlock(df, locs[k]) for k in locs}

## 2. Specify blocks:

### 2.1. Domestic production sector's split (```A-D```)

From the blocks we retrieve:
* Production sector index (```s_p```), domestic goods (```n_p```), foreign goods index (```n_F```), index for different tax categories (```taxTypes```). NB: The index ```n_Fother``` includes components that should be mapped to existing ```n_F```.
* Value of demand ```vD[s,n]``` for ```s_p``` and ```n_p```$\cup$ ```n_F```.
* Value of taxes ```vTax[s,taxTypes]``` for ```s_p``` + ```TotalTax[s]``` defined as the sum over ```taxTypes```.
* Value of demand ```vD[s,n]``` for ```s_p``` for ```n``` $=\lbrace$ L, resIncome $\rbrace$.

*A*

In [8]:
db['s_p'] = pd.Index(rowIndex.iloc[locs['P/In'][0][0]:locs['P/In'][0][1]], name = 's') # sectors from block A
db['n_p'] = db.get('s_p').rename('n') # goods from block A
blocks['P/In'].index = db.get('n_p') # add index to block A
blocks['P/In'].columns = db.get('s_p') # add index to block A

*B*

In [9]:
db['n_F'] = db.get('n_p')+'_F' # foreign goods index from block B
blocks['M/In'].index = db.get('n_F') # add index to block B
blocks['M/In'].columns = db.get('s_p') # add index to block B

*C*

In [10]:
db['n_Fother'] = pd.Index(rowIndex.iloc[locs['OT/In'][0][0]:locs['OT/In'][0][1]], name = 'n')
blocks['OT/In'].index = db.get('n_Fother')
blocks['OT/In'].columns = db.get('s_p')

*D*

In [11]:
D = pd.concat([blocks['PI/In'], blocks['TI/In']])
db['taxTypes'] = pd.Index(taxCategories, name = 'taxTypes')
D.index = pd.Index(rowIndex.iloc[locs['PI/In'][0][0]:locs['PI/In'][0][1]]).union(
          pd.Index(rowIndex.iloc[locs['TI/In'][0][0]:locs['TI/In'][0][1]]), sort = False).rename('temp')
D.columns = db.get('s_p')
db['vTax'] = adj.rc_pd(D, db.get('taxTypes').rename('temp')).rename_axis(index={'temp':'taxTypes'}).stack()
db['TotalTax'] = db.get('vTax').groupby('s').sum()
wages = D.xs(wageCategory) # wages
wages.index = pd.MultiIndex.from_product([pd.Index(['L'], name = 'n'), wages.index])
residualIncome = D.xs(residualIncomeCategory) # residual income
residualIncome.index = pd.MultiIndex.from_product([pd.Index(['resIncome'], name = 'n'), residualIncome.index])

*Collect:*

In [12]:
db['vD'] = pd.concat([blocks['P/In'].stack(), blocks['M/In'].stack(), blocks['OT/In'].stack(), wages,residualIncome])

### 2.2. Consumption (```E-H```)

From the blocks we retrieve:
* Indices: Aggregate household/government index ```s_HH```, ```s_G```. Index for government consumption ```gc```.
* The total value of private consumption, ```vD[n,HH]``` for ```n_p```$\cup$ ```n_F```.
* The value of government consumption ```vC[gc, n]``` for ```n_p```$\cup$ ```n_F```. 
    * In the Danish data, the index used in columns for government consumption is not unique. For instance, the code '06130' (indicating glasses, hearing aids, and similar) appears twice: Once under 'market valued public consumption' (MVPC) and once under 'non-market valued public consumption' (NMVPC). For now, we sum up government consumption to the transaction code level - thus not distinguishing between e.g. NMVPC and MVPC 
* The value of taxes split onto government consumption components: ```vC_tax[taxTypes, gc]```.
* The total value of government consumption, ```vD[n,G]``` for ```n_p```$\cup$ ```n_F```.
* Value of taxes ```vTax[s,taxTypes]``` for ```s_G``` + ```TotalTax[s]``` defined as the sum over ```taxTypes```.

**Private consumption:**

In [13]:
privateC = pd.concat([blocks['P/C'].iloc[:,0].set_axis(db.get('n_p')),
                      blocks['M/C'].iloc[:,0].set_axis(db.get('n_F')),
                      blocks['OT/C'].iloc[:,0].set_axis(db.get('n_Fother'))])
db['s_HH'] = pd.Index(['HH'], name = 's')
privateC.index = pd.MultiIndex.from_product([privateC.index, db.get('s_HH')])
db['vD'] = pd.concat([db.get('vD'), privateC])

Specify tax part:

In [14]:
blocks['PI/C'].index = pd.Index(rowIndex.iloc[locs['PI/C'][0][0]:locs['PI/C'][0][1]], name = 'taxTypes')
HHTax = blocks['PI/C'].iloc[:,0]
HHTax.index = pd.MultiIndex.from_product([HHTax.index, db.get('s_HH')])
db['vTax'] = HHTax.combine_first(db.get('vTax'))
db['TotalTax'] = HHTax.groupby('s').sum().combine_first(db.get('TotalTax'))

**Government consumption:**

In [23]:
db['s_G'] = pd.Index(['G'], name = 's')
gcomp = [c for c in cKeys if c.startswith('G_')] # components that identify government consumption
gc = pd.Index(colIndex.iloc[locs[f'P/{gcomp[0]}'][1][0]:locs[f'P/{gcomp[-1]}'][1][1]], name = 'gc') # government consumption set
gc_value = pd.concat([pd.concat([blocks[f'P/{g}'] for g in gcomp],axis=1),
                      pd.concat([blocks[f'M/{g}'] for g in gcomp],axis=1),
                      pd.concat([blocks[f'OT/{g}'] for g in gcomp],axis=1)], axis = 0)
gc_value.index = pd.Index(np.hstack([db.get(i) for i in ('n_p','n_F','n_Fother')]), name = 'n')
gc_value.columns = gc
db['gc'] = gc.unique()
db['vC'] = gc_value.stack().groupby(['n','gc']).sum()
gcTotal = gc_value.sum(axis=1)
gcTotal.index = pd.MultiIndex.from_product([gcTotal.index, db.get('s_G')])
db['vD'] = gcTotal.combine_first(db.get('vD'))

Specify tax part:

In [28]:
GovernmentTax = pd.concat([blocks[f'PI/{g}'] for g in gcomp],axis=1)
GovernmentTax.columns = gc
GovernmentTax.index = pd.Index(rowIndex.iloc[locs[f'PI/{gcomp[0]}'][0][0]:locs[f'PI/{gcomp[0]}'][0][1]], name = 'taxTypes')
db['vC_tax'] = GovernmentTax.stack().groupby(['taxTypes','gc']).sum()
GTax = GovernmentTax.sum(axis=1)
GTax.index = pd.MultiIndex.from_product([GTax.index, db.get('s_G')])
db['vTax'] = GTax.combine_first(db.get('vTax'))
db['TotalTax'] = GTax.groupby('s').sum().combine_first(db.get('TotalTax'))

### 2.3. Investments ```I-L```

From the blocks we retrieve:
* Indices: The subset ```s_i``` is a subset of ```s``` of sectors.   
* The demand from investment sectors ```vD[n,s]```.
* Value of taxes ```vTax[s,taxTypes]``` for ```s_i``` + ```TotalTax[s]``` defined as the sum over ```taxTypes```.

In [18]:
db['s_i'] = pd.Index(colIndex.iloc[locs['P/I'][1][0]:locs['P/I'][1][1]], name = 's').astype(str) # investment sectors
vD_inv = pd.concat([blocks[f'{x}/I'] for x in ('P','M','OT')], axis = 0)
vD_inv.columns = db.get('s_i')
vD_inv.index = pd.Index(np.hstack([db.get(i) for i in ('n_p','n_F','n_Fother')]), name = 'n')
db['vD'] = vD_inv.stack().combine_first(db.get('vD'))

Specify tax part:

In [19]:
blocks['PI/I'].index = pd.Index(rowIndex.iloc[locs['PI/I'][0][0]:locs['PI/I'][0][1]], name = 'taxTypes')
blocks['PI/I'].columns = db.get('s_i')
db['vTax'] = blocks['PI/I'].stack().combine_first(db.get('vTax'))
db['TotalTax'] = blocks['PI/I'].sum(axis=0).combine_first(db.get('TotalTax'))

### 2.4. Inventory invetments and exports ```M-P```

This part is a bit more ad-hoc.

In [20]:
db['s_f'] = pd.Index(['F'], name = 's')
M2P = pd.concat([blocks[f'{x}/Other'] for x in ('P','M','OT')], axis = 0)
M2P.columns = pd.Index(colIndex.iloc[locs['P/Other'][1][0]:locs['P/Other'][1][1]], name = 'temp').astype(str)
M2P.index = pd.Index(np.hstack([db.get(i) for i in ('n_p','n_F','n_Fother')]), name = 'n')
itory = M2P.loc[:, itoryCategories].sum(axis=1).rename('vD')
itory.index = pd.MultiIndex.from_product([itory.index, pd.Index(['itory'], name = 's')])
export= adj.rc_pd(M2P[exportCategory], db.get('n_p')).rename('vD')
export.index = pd.MultiIndex.from_product([export.index, db.get('s_f')])
db['vD'] = pd.concat([itory, export], axis = 0).combine_first(db.get('vD'))

Specify tax part:

In [21]:
blocks['PI/Other'].index = pd.Index(rowIndex.iloc[locs['PI/Other'][0][0]:locs['PI/Other'][0][1]], name = 'taxTypes')
blocks['PI/Other'].columns = M2P.columns
itory_tax = blocks['PI/Other'].loc[:,itoryCategories].sum(axis=1)
export_tax= blocks['PI/Other'][exportCategory]
export_tax.index = pd.MultiIndex.from_product([export_tax.index, db.get('s_f')])
itory_tax.index  = pd.MultiIndex.from_product([itory_tax.index, itory.index.levels[-1]])
db['vTax'] = pd.concat([itory_tax, export_tax], axis=0).combine_first(db.get('vTax'))
db['TotalTax'] = pd.concat([export_tax.groupby('s').sum(),itory_tax.groupby('s').sum()],axis=0).combine_first(db.get('TotalTax'))

## 3. Investment IO

For now, we only rely on the sums of investments goods. The IO data on investments, however, includes a full $(s \times s)$ input-output flows for (almost) each investment type. So, for now at least, we do the following:
* For each sheet (representing a single investment type - or a sum of multiple ones), we only extract the total value across the column sectors. This value is identified by a row indicator + columns.
* The investment data is only defined on a more aggregate level (69 sectors). We deal with this discrepancy later. 

*Only use sheets that don't start with 'told':*

In [22]:
r = 0
c = 1
rIndex = 3
rMarker = 'Investering i alt, købepriser'
cMarkers = ['Investerende brancher', 'Total']
def extractRow(df, rMarker, cMarkers, r = 0, c = 1, rIndex = 3):
    return pd.Series(df.iloc[getLoc(df.iloc[:,r], rMarker),getLoc(df.iloc[1,:], cMarkers[0]):getLoc(df.iloc[c,:], cMarkers[1])].values, index = pd.Index(df.iloc[rIndex,getLoc(df.iloc[1,:], cMarkers[0]):getLoc(df.iloc[c,:], cMarkers[1])], name = 's'))

In [23]:
sheets = {k: pd.DataFrame(wb_inv[k].values) for k in read.sheetnames_from_wb(wb_inv) if not k.startswith('told')}
db['vD_inv'] = pd.DataFrame({k: extractRow(sheets[k], rMarker, cMarkers) for k in sheets}).stack().rename_axis(index=['s','n'])

## 4. Durable IO

...